In [62]:
import syllapy
import numpy as np
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from readability import Readability
import pandas as pd
import re

In [2]:
import json
import csv
import random
import pandas as pd
import sklearn
import nltk
import torch
import torchvision
import tensorflow
import string
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from gensim import corpora, models
from sklearn.preprocessing import LabelEncoder

### Text Preparation

In [7]:
article_title = "Mysteries of the Deep, Scientists Uncover Bioluminescent Marvels in Uncharted Oceans"

In [8]:
news_article = """
In a groundbreaking discovery, scientists have identified a new species of marine life in the depths of the ocean. The species, named "Abyssal Glowfish," is known for its bioluminescent properties, creating a mesmerizing display of colors in the dark ocean depths.

Researchers conducted an extensive deep-sea expedition, using state-of-the-art submersibles to explore the previously uncharted regions. The Abyssal Glowfish was found thriving at depths of over 5,000 meters, adapting to the pitch-black environment with its unique ability to emit light.

This discovery opens up new avenues for understanding the biodiversity of our oceans, with estimates suggesting there could be millions of undiscovered species. The Abyssal Glowfish is just one example of the incredible life that may exist in these unexplored habitats.

Furthermore, environmentalists emphasize the importance of conservation efforts to protect these delicate ecosystems. The newfound species highlights the need for responsible stewardship of our oceans to preserve the rich diversity of marine life.

The scientific community is abuzz with anticipation for future deep-sea expeditions, hoping to unveil more secrets hidden beneath the ocean's surface. The Abyssal Glowfish serves as a symbol of the wonders yet to be discovered in the mysterious depths of our planet's oceans.
"""


In [9]:
cleaned_article = ' '.join(news_article.split()).replace("\'", '')

### Flesch-Kincaid Grade Level Formula

Fleisch readibility score should be between 8-12
If score is above 12, take score - 12 = diff and do 100 - (diff * 10)
If score is below 8, take 8 - score = diff and do 100 - (diff * 10)

Readibility is very important because it has to do with the credibility of the author. A good author should know that the average american is high school educated, and that they should aim for a reading grade level between 8-12 based on good journalism practice. Ignoring this hinders reliability unless they work for a specialized news source that caters towards a more educated audience. If this is true though, the other scoring metrics should make up for the reduction from this metric.

formula = 0.39 * (total words / total sentences) + 11.8 (total syllables / total words) - 15.59

In [21]:
r = Readability(cleaned_article)

In [22]:
fk = r.flesch_kincaid()

In [23]:
flesch_score = fk.score

In [24]:
if flesch_score > 12:
    diff = flesch_score - 12
    fk_rating = 100 - (diff * 10)
elif flesch_score < 8:
    diff = 8 - flesch_score
    fk_rating = 100 - (diff * 10)

In [25]:
fk_rating

75.54633165829148

### Sensationalism

In [15]:
sia = SentimentIntensityAnalyzer()
moving_sentiment_value = 0
number_of_paragraphs = 0
paragraphs = news_article.split('\n\n')
for i in paragraphs:
    cleaned_text = ' '.join(i.split()).replace("\'", '')
    compound_sentiment_score = sia.polarity_scores(cleaned_text)['compound']
    moving_sentiment_value += compound_sentiment_score
    number_of_paragraphs += 1

In [16]:
overall_sentiment = moving_sentiment_value / number_of_paragraphs
overall_sentiment

0.39776

In [18]:
overall_sent_score = 100
if overall_sentiment < -0.2:
 overall_sent_score = 100 + (overall_sentiment * 100)

In [20]:
overall_sent_score

100

### Clickbait

In [27]:
def predict_label(confidence):
    if confidence < 0.166:
        return "pants-fire"
    elif confidence < 0.33:
        return "false"
    elif confidence < 0.5:
        return "barely-true"
    elif confidence < 0.666:
        return "half-true"
    elif confidence < 0.833:
        return "mostly-true" 
    else:
        return "true"

In [28]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    return text

In [29]:
def get_ngram_embeddings(text, n, model):
    words = text.split()
    ngrams = [words[i:i + n] for i in range(len(words) - n + 1)]  
    embeddings = [model.wv[gram] for gram in ngrams if all(word in model.wv for word in gram)]
    return embeddings

In [30]:
def get_word_embeddings(tokens, model):
    embeddings = [model.wv[word] for word in tokens if word in model.wv]
    return np.mean(embeddings, axis=0) if embeddings else np.zeros(model.vector_size)

In [31]:
def predict_label(confidence):
    if confidence < 0.166:
        return "pants-fire"
    elif confidence < 0.33:
        return "false"
    elif confidence < 0.5:
        return "barely-true"
    elif confidence < 0.666:
        return "half-true"
    elif confidence < 0.833:
        return "mostly-true" 
    else:
        return "true"

In [32]:
clickbait = pd.read_csv("Data/Clickbait_Data/clickbait_data.csv")

In [33]:
#determining clickbait
names = ["Linear SVM"] 

classifiers = [SVC(kernel="linear", C=0.025, probability=True)]


#Preprocess, train/test split, and 
clickbait['PreprocessedTitle'] = clickbait['headline'].apply(preprocess_text)
X_train_click, X_test_click, y_train_click, y_test_click = train_test_split(clickbait['PreprocessedTitle'], clickbait['clickbait'], test_size=0.2, random_state=42)
count_vectorizer = CountVectorizer()
X_train_counts = count_vectorizer.fit_transform(X_train_click)
X_test_counts = count_vectorizer.transform(X_test_click)


max_score = 0.0
max_class = ''
# iterate over classifiers
for name, clf in zip(names, classifiers):
    start_time = time.time()
    clf.fit(X_train_counts, y_train_click)
    score = 100.0 * clf.score(X_test_counts, y_test_click)
    print('Classifier = %s, Score (test, accuracy) = %.2f,' %(name, score), 'Training time = %.2f seconds' % (time.time() - start_time))
    
    if score > max_score:
        clf_best = clf
        max_score = score
        max_class = name

print(80*'-' )
print('Best --> Classifier = %s, Score (test, accuracy) = %.2f' %(max_class, max_score))

Classifier = Linear SVM, Score (test, accuracy) = 95.67, Training time = 52.63 seconds
--------------------------------------------------------------------------------
Best --> Classifier = Linear SVM, Score (test, accuracy) = 95.67


In [34]:
article_title_processed = preprocess_text(article_title)
article_title_vectorized = count_vectorizer.transform([article_title_processed])
clickbait_probability = clf_best.predict_proba(article_title_vectorized)

In [39]:
confidence_not_clickbait = clickbait_probability[:, 0]
confidence_not_clickbait = confidence_not_clickbait[0] * 100
confidence_not_clickbait

95.6163109881357

### Dashboard

make dashboard with outcome of each factor and apply appropriate weighting for each factor. After that read up on RAG/LangChain article

In [3]:
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output

/var/folders/pb/7hrkp8tj05gbr513hzd6bv000000gp/T/ipykernel_3336/488439558.py:2: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/var/folders/pb/7hrkp8tj05gbr513hzd6bv000000gp/T/ipykernel_3336/488439558.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [61]:
# Sample data (replace with your actual data)
not_clickbait_confidence = confidence_not_clickbait
sensationalism_score = overall_sent_score
readability = fk_rating
overall_score = (not_clickbait_confidence + sensationalism_score + readability) / 3

# Initialize Dash app
app = dash.Dash(__name__)

# Define layout
app.layout = html.Div([
    html.H1("News Article Truthfulness Dashboard", style={'text-align': 'center'}),
    
    # Display selected data
    html.Div([
        html.H3("Examined Article: " + article_title, style={'text-align': 'center'}),
        html.Pre(f"Clickbait Score: {confidence_not_clickbait}"),
        html.Pre(f"Sensationalism Score: {overall_sent_score}"),
        html.Pre(f"Readability Score: {fk_rating}"),
        html.Pre(f"Overall Truthfulness Score: {overall_score}")
    ], style={'text-align': 'center'})
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)